# Diversity and Inclusion Company Score Card
![DnI Image](https://gresb.com/wp-content/uploads/2019/08/People_Graphic-768x398.jpg)

In [1]:
import eikon as ek
import plotly.offline as offline
from plotly import graph_objs as go
from IPython.display import display, Markdown
offline.offline.init_notebook_mode(connected=True)

In [2]:
ek.set_app_key('8a0a51d096f34eec86dadab1763ad94dae81c80e')
symbol = 'MSFT.O'

## Company fundamental information

In [3]:
d1, err = ek.get_data(symbol, fields = ['TR.CompanyName', 'TR.TRBCIndustryGroup', 'TR.HeadquartersCountry', 'TR.TRBCIndustryGroupCode', 'TR.HQCountryCode'])
inst = {
    'RIC': symbol,
    'Name': d1.iloc[0][1],
    'IndustryGroup': d1.iloc[0][2],
    'Headquarters': d1.iloc[0][3],
    'IndustryCode': d1.iloc[0][4],
    'CountryCode': d1.iloc[0][5],
}

for key, val in inst.items():
    display(Markdown('**{}:** {}'.format(key, val)))

**RIC:** MSFT.O

**Name:** Microsoft Corp

**IndustryGroup:** Software & IT Services

**Headquarters:** United States of America

**IndustryCode:** 572010

**CountryCode:** US

## Use screener to get country and industry peers

In [4]:
# define the ranking function
rFields = ['TR.CommonName', 'TR.TRDIRScore']
def getRank(screenerQuery):
    iall, err = ek.get_data(instruments = screenerQuery, fields = rFields)
    iall_sorted = iall.sort_values('TRDIR Score', ascending=0).reset_index(drop=True)
    iRank = iall_sorted.loc[iall_sorted['Company Common Name'] == inst['Name']].index[0]
    sRank = str(iRank + 1) + "/" + str(iall_sorted.index.max() + 1)
    return sRank, iall_sorted

In [5]:
# Get overall rank
overallRank, overallList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, CURN=USD)')
display(Markdown('**Overall Rank:** {}'.format(overallRank)))

# Get country rank
countryRank, countryList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, IN(TR.HQCountryCode,' + inst['CountryCode'] + '), CURN=USD)')
display(Markdown('**Country Rank:** {}'.format(countryRank)))

# Get industry rank
industryRank, industryList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, IN(TR.TRBCIndustryGroupCode,' + str(inst['IndustryCode']) + '), CURN=USD)')
display(Markdown('**Industry Group Rank:** {}'.format(industryRank)))

**Overall Rank:** 351/2732

**Country Rank:** 61/579

**Industry Group Rank:** 13/124

## Top and Bottom 5 in this industry

In [6]:
industryList[['Company Common Name', 'TRDIR Score']].iloc[0: 5]

,Company Common Name,TRDIR Score
0,Accenture PLC,83.75
1,SAP SE,70.75
2,NortonLifeLock Inc,70.00
3,Fujitsu Ltd,69.25
4,Salesforce.Com Inc,69.00


In [7]:
industryList[['Company Common Name', 'TRDIR Score']].iloc[-5: industryList.shape[0]]

,Company Common Name,TRDIR Score
119,Cars.com Inc,39.75
120,Paylocity Holding Corp,38.50
121,Alarm.com Holdings Inc,38.50
122,Ebix Inc,37.50
123,Colopl Inc,33.50


## Comparison with Country and Industry benchmarks

In [8]:
aFields = ['TR.CommonName', 'TR.TRDIRScore', 'TR.TRDIRDiversityScore', 'TR.TRDIRPeopleDevelopmentScore', 'TR.TRDIRInclusionScore', 'TR.TRDIRControvScore']
bestInIndustry = industryList.iloc[0]['Instrument']
bestInCountry = countryList.iloc[0]['Instrument']
dnIdata, err = ek.get_data(instruments = [symbol, bestInCountry, bestInIndustry], fields = aFields)
compare = dnIdata.transpose()
compare.columns = ['', 'Top Ranked {}'.format(inst['CountryCode']), inst['IndustryGroup']]
compare

,,Top Ranked US,Software & IT Services
Instrument,MSFT.O,OC.N,ACN.N
Company Common Name,Microsoft Corp,Owens Corning,Accenture PLC
TRDIR Score,62.75,77.25,83.75
TRDIR Diversity Score,56,52,67
TRDIR People Development Score,57,65,75
TRDIR Inclusion Score,50,92,93
TRDIR Controversies Score,88,100,100


In [9]:
# Plot it in the bar chart
def generateBar(index):
    cName = compare.columns[index]
    return go.Bar(x=compare.index[2:], y=compare[cName][2:], name=compare[cName]['Company Common Name'])

fig = go.Figure([generateBar(0), generateBar(1), generateBar(2)])
fig.update_layout(title='Best in Class Score Comparison')
fig.show()

In [10]:
dFields = ['TR.AnalyticBoardCulturalDiversity(IncludePartialYear=False)', 'TR.CGAnalyticBoardFemale(IncludePartialYear=False)', 'TR.AnalyticExecutiveMembersGenderDiversity(IncludePartialYear=False)', 'TR.TargetsDiversityOpportunity(IncludePartialYear=False)', 'TR.PolicyDiversityOpportunity(IncludePartialYear=False)', 'TR.WomenEmployees(IncludePartialYear=False)', 'TR.WomenManagers(IncludePartialYear=False)', 'TR.NewWomenEmployees(IncludePartialYear=False)']
ddata, err = ek.get_data(instruments = [symbol, bestInCountry, bestInIndustry], fields = dFields)
compare = ddata.transpose()
compare.columns = ['', 'Top Ranked {}'.format(inst['CountryCode']), inst['IndustryGroup']]
display(Markdown('### DIVERSITY PILLAR SCORES'))
display(compare)

### DIVERSITY PILLAR SCORES

,,Top Ranked US,Software & IT Services
Instrument,MSFT.O,OC.N,ACN.N
"Board Cultural Diversity, Percent",28.5714,8.33333,41.6667
Female on Board,28.5714,25,33.3333
"Executive Members Gender Diversity, Percent",37.5,30,14.2857
Targets Diversity and Opportunity,True,True,True
Policy Diversity and Opportunity,True,True,True
Women Employees,26.6,18.42,42
Women Managers,19.7,23.79,29
New Women Employees,NaN,20.255,47


In [11]:
iFields = ['TR.EmployeeSatisfaction(IncludePartialYear=False)', 'TR.PolicyCareerDevelopment', 'TR.AvgTrainingHours', 'TR.InternalPromotion', 'TR.MgtTraining', 'TR.AnalyticTrainingCosts', 'TR.PolicySkillsTraining']
idata, err = ek.get_data(instruments = [symbol, bestInCountry, bestInIndustry], fields = iFields)
compare = idata.transpose()
compare.columns = ['', 'Top Ranked {}'.format(inst['CountryCode']), inst['IndustryGroup']]
display(Markdown('### INCLUSION PILLAR SCORES'))
display(compare)

### INCLUSION PILLAR SCORES

,,Top Ranked US,Software & IT Services
Instrument,MSFT.O,OC.N,ACN.N
Employee Satisfaction,NaN,97,NaN
Policy Career Development,True,True,True
Average Training Hours,NaN,6.93,42
Internal Promotion,True,False,True
Management Training,True,True,True
Training Costs Per Employee,NaN,NaN,2020.66
Policy Skills Training,True,True,True
